# 03. EDS biasing using HTF

Here the collective variable (CV) being biased is the average distance to center of mass.

In [1]:
import hoomd
import hoomd.md
import hoomd.htf as htf
import tensorflow as tf

In [2]:
#### Build training graph ####

def make_eds_graph(N, NN, set_pt):
    # N= Number of atoms in the system 
    # NN=Number of nearest neighbors 
    # set_pt=set point in EDS method
    graph =htf.graph_builder(NN,output_forces=True)
    #calculate center of mass
    com = tf.reduce_mean(graph.positions[:, :2], 0) 
    #calculate distance of each atom from center of mass
    rs = graph.safe_norm(tf.math.subtract(graph.positions[:, :2], com), axis=1) 
    #calculate the average distance from center of mass. This is the collective variable (CV)
    real_cv = tf.reduce_mean(rs) 
    #calculates the running mean of the CV
    cv_run=graph.running_mean(tensor=real_cv,name='cv_run') 
    #calculate the EDS alpha value every 300 steps. 
    eds_alpha = htf.eds_bias(real_cv, set_point=set_pt, period=300,learning_rate=5.0)
    eds_energy = eds_alpha * real_cv #computes EDS energy
    #compute EDS forces
    eds_forces = graph.compute_forces(eds_energy)
    printer1=tf.Print(cv_run,[cv_run],message='CV')
    printer2=tf.Print(eds_alpha,[eds_alpha],message='a')
    graph.save('my_model',out_nodes=[[printer1,100],[printer2,100]],force_tensor=eds_forces,virial=None)


In [3]:
#### Hoomd-Sim code ####

make_eds_graph(64, 100, 4.0)

hoomd.context.initialize("--mode=cpu")
with htf.tfcompute('my_model', device='CPU:0') as tfcompute:
    #cut off radius: must be less than the box size
    rcut = 6.0 
    #initialize the lattice
    system = hoomd.init.create_lattice(unitcell=hoomd.lattice.sq(a=2.0),n=[8, 8])
    nlist = hoomd.md.nlist.cell(check_period=1)
    #enable lj pair potential
    lj = hoomd.md.pair.lj(rcut, nlist) 
    #set lj coefficients
    lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0) 
    hoomd.md.integrate.mode_standard(dt=0.005)
    # set up NVT simulation
    hoomd.md.integrate.nvt(kT=1.0, tau=0.5,group=hoomd.group.all()) 
    #equilibrate
    hoomd.run(3000)
    #simulation                  
    tfcompute.attach(nlist, r_cut=rcut)
    hoomd.run(10000)

print('Training Done')






Instructions for updating:
Use tf.print instead of tf.Print. Note that tf.print returns a no-output operator that directly prints the output. Outside of defuns or eager mode, this operator will not be executed unless it is directly specified in session.run or used as a control dependency for other operators. This is only a concern in graph mode. Below is an example of how to ensure tf.print executes in graph mode:


HOOMD-blue v2.5.1 CUDA (10.0) SINGLE SSE SSE2 
Compiled: 03/07/2020
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Compute